In [1]:
import re
import nrrd
import os, glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

import loss
import Helper
import allMetrics

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')

In [ ]:
physical_devices

In [ ]:
try:
  for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [8]:
access_token = '5307815831:AAEKVzHQFAVWMwXseU9L1yKKtsBq4YPZM8Y'
chat_id = '503932462'

# data_dir = '/home/tester/jianhoong/jh_fyp_work/ct_scans_data/raw_data/' # ntu
data_dir = '/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/raw_data/' # dgx
# data_dir = 'E:/FYP/jh_fyp_work/ct_scans_data/raw_data/'  # Home pc

z_train = os.path.join(data_dir, 'train_data')
z_train_image = os.path.join(z_train, 'training_images')
z_train_mask = os.path.join(z_train, 'training_masks')

z_valid = os.path.join(data_dir, 'valid_data')
z_valid_image = os.path.join(z_valid, 'valid_images')
z_valid_mask = os.path.join(z_valid, 'valid_masks')

z_test = os.path.join(data_dir, 'test_data')
z_test_image = os.path.join(z_test, 'testing_images')
z_test_mask = os.path.join(z_test, 'testing_masks')

In [9]:
def sorted_alnum(l):
    def convert(text): return int(text) if text.isdigit() else text
    def alphanum_key(key): return [convert(c)
                                   for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)


In [15]:
# train_data_with_96 = [15, 16, 20, 21, 23, 33, 37, 52, 64, 69, 72, 76, 77, 83, 92, 94, 95,
#                       97, 103, 116, 123, 129, 157, 159, 162, 163, 170, 171, 180, 191, 192, 198, 200, 207, 209]
# valid_data_with_96 = [213, 219, 221, 227, 231,
#                       238, 240, 243, 249, 250, 255, 256, 257, 270]
# test_data_with_96 = [271, 272, 274, 277, 283,
#                      286, 287, 293, 296, 300, 303, 305, 306, 314]

train_data_with_96 = [17, 18, 19, 22, 24, 25, 26, 27, 28]
valid_data_with_96 = [213, 219, 221, 227, 231,
                      238, 240, 243, 249, 250, 255, 256, 257, 270]
test_data_with_96 = [271, 272, 274, 277, 283,
                     286, 287, 293, 296, 300, 303, 305, 306, 314]

train_path = sorted_alnum([os.path.join(z_train_image, file) for file in os.listdir(
    z_train_image) if int(re.findall(r'\d+', file)[0]) in train_data_with_96])
train_mask_path = sorted_alnum([os.path.join(z_train_mask, file) for file in os.listdir(
    z_train_mask) if int(re.findall(r'\d+', file)[0]) in train_data_with_96])

valid_path = sorted_alnum([os.path.join(z_valid_image, file) for file in os.listdir(
    z_valid_image) if int(re.findall(r'\d+', file)[0]) in valid_data_with_96])
valid_mask_path = sorted_alnum([os.path.join(z_valid_mask, file) for file in os.listdir(
    z_valid_mask) if int(re.findall(r'\d+', file)[0]) in valid_data_with_96])

test_path = sorted_alnum([os.path.join(z_test_image, file) for file in os.listdir(
    z_test_image) if int(re.findall(r'\d+', file)[0]) in test_data_with_96])
test_mask_path = sorted_alnum([os.path.join(z_test_mask, file) for file in os.listdir(
    z_test_mask) if int(re.findall(r'\d+', file)[0]) in test_data_with_96])

In [13]:
# /media/dro/JHSeagate/FYP/ # dgx
# /home/tester/jianhoong/ # ntu
# E:/FYP/ # home

model_checkpoint_path = '/media/dro/JHSeagate/FYP/jh_fyp_work/3D_UNet/Final/SavedModels/Model8.hdf5'


model = tf.keras.models.load_model(model_checkpoint_path, custom_objects = {
                                                                                   'dice_coef':allMetrics.dice_coef,
                                                                                   'tversky_crossentropy':loss.tversky_crossentropy})

2023-05-22 10:40:03.428576: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 10:40:03.901982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38414 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


In [17]:
train_path[1]

'/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/raw_data/train_data/training_images/P18_image.nrrd'

In [20]:
input_shape = (64,64,96)
num_class = 1
desired_step = 32

test_patient_img = Helper.process_scan(train_path[4])
test_patient_msk = Helper.read_nrrd_file(train_mask_path[4])

test_patch = patchify(test_patient_img, input_shape, step = desired_step) # Yield 15 x 15 x 3 of size 64 x 64 x 32 cubes
test_mask_patch = patchify(test_patient_msk, input_shape, step = desired_step)

test_patient_cubes = np.reshape(test_patch, (-1, test_patch.shape[3], test_patch.shape[4], test_patch.shape[5])) # Collapse 15 x 15 x 3 into 675 cubes x 64 x 64 x 32
test_patient_mask_cubes = np.reshape(test_mask_patch, (-1, test_mask_patch.shape[3], test_mask_patch.shape[4], test_mask_patch.shape[5])) # 675 x 64 x 64 x 32

test_patient_input = np.expand_dims(test_patient_cubes, axis=4)
test_patient_mask_input = np.expand_dims(test_patient_mask_cubes, axis=4)

test_patient_pred = model.predict(test_patient_input)

test_patient_pred_reshaped = np.reshape(test_patient_pred, 
                                        (test_patch.shape[0], test_patch.shape[1], test_patch.shape[2],
                                         test_patch.shape[3], test_patch.shape[4], test_patch.shape[5]))

reconstruct_pred = unpatchify(test_patient_pred_reshaped, test_patient_img.shape)
# reconstruct_pred[reconstruct_pred < 45] = 0
# reconstruct_pred[reconstruct_pred > 50] = 255



@interact
def explore_prediction(layer = (0,96), view = ["axial"]):
    if view == 'axial':
        img_array_view = test_patient_img[:, :, layer]
        msk_array_view = test_patient_msk[:, :, layer]
        pred_array_view = reconstruct_pred[:, :, layer]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=48, description='layer', max=96), Dropdown(description='view', options=(…

In [21]:
@interact
def explore_prediction(layer = (0,511), view = ["coronal"]):
    if view == 'coronal':
        img_array_view = test_patient_img[:, layer, :]
        msk_array_view = test_patient_msk[:, layer, :]
        pred_array_view = reconstruct_pred[:, layer, :]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()
    
@interact
def explore_prediction(layer = (0,511), view = ["sagittal"]):
    if view == 'sagittal':
        img_array_view = test_patient_img[layer, :, :]
        msk_array_view = test_patient_msk[layer, :, :]
        pred_array_view = reconstruct_pred[layer, : , :]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=255, description='layer', max=511), Dropdown(description='view', options…

interactive(children=(IntSlider(value=255, description='layer', max=511), Dropdown(description='view', options…

In [22]:
headerInfo = nrrd.read_header(train_path[4])

In [23]:

pred_volume_file_location = f"/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/prediction_data/Model8_P24.nrrd"
nrrd.write(pred_volume_file_location, reconstruct_pred, headerInfo)

In [ ]:
def generate_reconstruct_pred(patient_path):
    
    headerInfo = nrrd.read_header(patient_path)
    
    patient_idx = int(re.findall(r'\d+', patient_path[-14:-11])[0])
    pred_volume_file_location = f"/media/dro/JHSeagate/FYP/jh_fyp_work/ct_scans_data/prediction_data/Model_9(DGX)_Predictions/P{patient_idx}_pred_Model9.nrrd"

    
    input_shape = (64,64,32)
    num_class = 1
    desired_step = 32
    
    test_patient_img = Helper.process_scan(patient_path)
    test_patient_msk = Helper.read_nrrd_file(patient_path)

    test_patch = patchify(test_patient_img, input_shape, step = desired_step) # Yield 15 x 15 x 3 of size 64 x 64 x 32 cubes
    test_mask_patch = patchify(test_patient_msk, input_shape, step = desired_step)

    test_patient_cubes = np.reshape(test_patch, (-1, test_patch.shape[3], test_patch.shape[4], test_patch.shape[5])) # Collapse 15 x 15 x 3 into 675 cubes x 64 x 64 x 32
    test_patient_mask_cubes = np.reshape(test_mask_patch, (-1, test_mask_patch.shape[3], test_mask_patch.shape[4], test_mask_patch.shape[5])) # 675 x 64 x 64 x 32

    test_patient_input = np.expand_dims(test_patient_cubes, axis=4)
    test_patient_mask_input = np.expand_dims(test_patient_mask_cubes, axis=4)

    test_patient_pred = model.predict(test_patient_input)

    test_patient_pred_reshaped = np.reshape(test_patient_pred, 
                                            (test_patch.shape[0], test_patch.shape[1], test_patch.shape[2],
                                             test_patch.shape[3], test_patch.shape[4], test_patch.shape[5]))

    reconstruct_pred = unpatchify(test_patient_pred_reshaped, test_patient_img.shape)
    nrrd.write(pred_volume_file_location, reconstruct_pred, headerInfo)
    
    return


In [ ]:
for file_path in train_path:
    generate_reconstruct_pred(file_path)